# **Lexicon**

In [1]:
import numpy as np
import pandas as pd

### Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### DB 연결
# !pip install pymysql
import pymysql

### DB에 저장
import sqlalchemy
from sqlalchemy import create_engine

### 실시간 주식가격 데이터
# !pip install finance-datareader
import FinanceDataReader as fdr

### 텍스트 분석
## KoNLPy
# 1) JAVA 설치, 2) Python 버전과 맞는 JPype1-py3 설치, 3) !pip install konlpy, 4) 설치 경로에서 jvm.py 파일 코드 67번 줄 주석 처리 
from konlpy.tag import Okt
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
## FastText
# !pip install gensim
# !pip install fasttext
import fasttext
import fasttext.util
# Facebook 한국어 Embedding 모델 다운로드 → 한 번만 설치하면 됨
fasttext.util.download_model('ko', if_exists='ignore')
# 유사도 계산
from gensim import models

### 모델 학습 및 평가
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import precision_score, recall_score, f1_score

### 모델 저장 및 로드
import joblib

### 기타
import datetime
from collections import Counter
import sys
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf

print('GPU', '사용 가능' if tf.config.experimental.list_physical_devices('GPU') else '사용 불가능')

GPU 사용 불가능


In [3]:
!nvidia-smi

Sun Sep 19 01:51:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 452.56       Driver Version: 452.56       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce MX450      WDDM  | 00000000:2D:00.0 Off |                  N/A |
| N/A   61C    P8    N/A /  N/A |    119MiB /  2048MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## **Read KOSELF & Test Data**

### **① KOSELF 감성 어휘 사전**

In [4]:
# 블로그에서 가져온 기본적인 한국어 긍부정 텍스트 목록
with open('positive_words_self.txt', encoding='utf-8') as pos_blog:
    positive_blog = pos_blog.readlines()
positive_blog = [pos_blog.replace('\n', '') for pos_blog in positive_blog]
with open('negative_words_self.txt', encoding='utf-8') as neg_blog:
    negative_blog = neg_blog.readlines()
negative_blog = [neg_blog.replace('\n', '') for neg_blog in negative_blog]

# KOSELF 감성 어휘 사전
with open('KOSELF_pos.txt', encoding='utf-8') as pos:
    positive = pos.readlines()
positive = [pos.replace('\n', '') for pos in positive]
with open('KOSELF_neg.txt', encoding='utf-8') as neg:
    negative = neg.readlines()
negative = [neg.replace('\n', '') for neg in negative]

### **② 2021년 Hyundai Data**

In [5]:
lexicon_2021 = pd.read_csv('../../../../Code/Data/Test/Stock-Year/2021_hyundai.csv')

lexicon_2021.head(1)

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization
0,현대차,5380,매일경제,2021010112,[풀영상] 2021 영동대로 밤하늘을 자동차가 날았다,http://news.mk.co.kr/newsRead.php?no=1330&year...,\n\n\n\n▶ 여기를 누르시면 크게 보실 수 있습니다\n 현대자동차가 서울 강...,2021-01-04,12,194000,211000,193500,207500,5651695,0.080729,1,1,여기 크게 자동차 서울 강남 영동대로 촬영 신축년 환영 드론 영상 공개 현대차 지난...


In [6]:
lexicon_2021[lexicon_2021['text'].isna()]

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization


In [7]:
lexicon_2021.dropna(axis=0, inplace=True)
lexicon_2021 = lexicon_2021.reset_index(drop=True)
lexicon_2021.isna().sum()

st_n                 0
st_cd                0
news                 0
datetime             0
title                0
url                  0
text                 0
date                 0
time                 0
Open                 0
High                 0
Low                  0
Close                0
Volume               0
Change               0
UpDown               0
Extremely_Changed    0
Tokenization         0
dtype: int64

## **Calculate Cosine Similarity**

In [8]:
# 이미 학습된 FastText 내장 한국어 모델
try:
    print(ko_model, '모델 로드 불필요')
except:
    ko_model = models.fasttext.load_facebook_model('cc.ko.300.bin')

In [9]:
limit_number = 15000
sys.setrecursionlimit(limit_number)

In [ ]:
pos_lexicon_2021 = pd.DataFrame(columns=['date', 'news_num', 'KOSELF_pos_word', 'news_word', 'cosine_similarity', 'frequency'])   # 'frequency' 구현할 방법은 추후 생각...ㅠㅠ

# KOSELF_pos와의 Cosine Similarity 계산
for x in range(len(lexicon_2021['Tokenization'])):
    news_num = x+1
    for y in range(len(positive)):
        for z in range(len(list(set(lexicon_2021['Tokenization'][x].split())))):
            if (ko_model.wv.similarity(positive[y], list(set(lexicon_2021['Tokenization'][x].split()))[z]) >= 0.5) and (ko_model.wv.similarity(positive[y], list(set(lexicon_2021['Tokenization'][x].split()))[z]) != 1.0):
                freq = 0
                for w in range(len(lexicon_2021['Tokenization'][x].split())):
                    if lexicon_2021['Tokenization'][x].split()[w] == list(set(lexicon_2021['Tokenization'][x].split()))[z]:
                        freq += 1
                data = {
                    'date': lexicon_2021['date'][x],
                    'news_num': news_num,
                    'KOSELF_pos_word': positive[y],
                    'news_word': list(set(lexicon_2021['Tokenization'][x].split()))[z],
                    'cosine_similarity': ko_model.wv.similarity(positive[y], list(set(lexicon_2021['Tokenization'][x].split()))[z]),
                    'frequency': freq
                    }
                pos_lexicon_2021 = pos_lexicon_2021.append(data, ignore_index=True)
                print("***{0} Cosine Similarity between <{1}> & <{2}> : {3}".format(lexicon_2021['date'][x], positive[y], list(set(lexicon_2021['Tokenization'][x].split()))[z], ko_model.wv.similarity(positive[y], list(set(lexicon_2021['Tokenization'][x].split()))[z])))

***2021-01-04 Cosine Similarity between <본격적인> & <본격> : 0.6104379892349243
***2021-01-04 Cosine Similarity between <증대> & <확대> : 0.5193238258361816
***2021-01-04 Cosine Similarity between <추진> & <진행> : 0.5358568429946899
***2021-01-04 Cosine Similarity between <추진> & <사업> : 0.5213737487792969
***2021-01-04 Cosine Similarity between <추진> & <추진> : 0.9999999403953552
***2021-01-04 Cosine Similarity between <증대> & <확대> : 0.5193238258361816
***2021-01-04 Cosine Similarity between <증대> & <확보> : 0.5161436796188354
***2021-01-04 Cosine Similarity between <충족> & <요구> : 0.5395625233650208
***2021-01-04 Cosine Similarity between <해결> & <문제> : 0.6186186075210571
***2021-01-04 Cosine Similarity between <해결> & <방법> : 0.5185782313346863
***2021-01-04 Cosine Similarity between <증대> & <증가> : 0.6946426630020142
***2021-01-04 Cosine Similarity between <증대> & <확대> : 0.5193238258361816
***2021-01-04 Cosine Similarity between <추진> & <계획> : 0.5205424427986145
***2021-01-04 Cosine Similarity between <중요한> & <

In [16]:
pos_lexicon_2021

,date,news_num,KOSELF_pos_word,news_word,cosine_similarity,frequency
0,2021-01-04,2,본격적인,본격,0.610438,1
1,2021-01-04,2,증대,확대,0.519324,1
2,2021-01-04,2,추진,진행,0.535857,1
3,2021-01-04,2,추진,사업,0.521374,2
4,2021-01-04,2,추진,추진,1.000000,1
...,...,...,...,...,...,...
17425,2021-09-16,3381,장점,특징,0.644235,1
17426,2021-09-16,3381,증대,증가,0.694643,1
17427,2021-09-16,3381,증대,확대,0.519324,2
17428,2021-09-16,3381,추진,계획,0.520542,1


In [17]:
pos_lexicon_2021.to_csv('../../../../Code/Data/Test/Stock-Year/pos_hyundai_2021.csv', index=False)

In [18]:
a = list(set(list(pos_lexicon_2021['news_word'])))

a

['횡보할',
 '중요',
 '인국공',
 '소재효',
 '잉스커츠',
 '폭락',
 '뤼디스페데',
 '압착설비',
 '진행',
 '훈풍',
 '기지원',
 '인컨설팅',
 '넥쏘향',
 '배준혁',
 '디지털리서치',
 '아임이',
 '추동훈',
 '수펙스',
 '사업',
 '스트럿링',
 '캥스터즈',
 '산화칼륨',
 '부카티',
 '흐트러지',
 '간펑리튬',
 '투무브',
 '잔손질',
 '법정상속',
 '가맹업',
 '어탭티브',
 '윤자민',
 '저하',
 '샤리오',
 '약세',
 '택시엽합회',
 '침체',
 '자공시',
 '레이텀앤왓킨스',
 '래몽래',
 '김교영',
 '강점',
 '알라배마',
 '단점',
 '웃돈',
 '극복',
 '우호',
 '렌탈식',
 '스베르도르프',
 '아뇨스',
 '보급물량',
 '후판값',
 '증대',
 '텍오스코텍',
 '가팔랐으',
 '특징',
 '겹악재',
 '그레주',
 '정조종사',
 '철도기지',
 '소형모듈원전',
 '급락',
 '재택생활',
 '계획',
 '진압되',
 '박학주',
 '코람코',
 '이중효과',
 '콘셉티카',
 '에어덕트',
 '기록',
 '배팅했다',
 '엔드플레이트',
 '펜데믹',
 '불황',
 '머니무브',
 '박제완',
 '카투홈',
 '최고점',
 '리레이팅',
 '짜릿거리',
 '웃돌',
 '문제',
 '레그단비맙',
 '운동장역',
 '크래프톤크래프톤',
 '활황',
 '배급회사',
 '쟁대위',
 '전개',
 '확립',
 '꿀렁임',
 '최고급',
 '스폰되',
 '폐쇄되',
 '향상',
 '이점',
 '본격',
 '얼티움셀즈가',
 '적극',
 '펠로들',
 '뉴호라이즌스',
 '김한엽',
 '지성규',
 '박유악',
 '덴티움덴티움',
 '호캉스',
 '커넥티브',
 '뉴인텍뉴인텍',
 '몰딩류',
 '발전',
 '증가',
 '임희진',
 '방안',
 '펏스원',
 '박기효',
 '카포테인먼트',
 '특성',
 '광풍',
 '에스오에스',
 '소다라',
 

In [19]:
b = list(set(list(pos_lexicon_2021[pos_lexicon_2021['cosine_similarity']>=0.7]['news_word'])))

b

['밑돈', '단점', '추진', '약세', '불황', '우세', '최고점', '실패', '추천']

In [20]:
c = list(set(list(pos_lexicon_2021[pos_lexicon_2021['cosine_similarity']>=0.65]['news_word'])))

c

['에스오에스',
 '추천',
 '것이므',
 '스타벤처스',
 '불황',
 '최고점',
 '감소',
 '웃돌',
 '송출되',
 '배급회사',
 '약세',
 '우세',
 '실린더헤드',
 '밑돈',
 '적극',
 '단점',
 '추진',
 '지성규',
 '실패',
 '증가']